In [ ]:
!pip install openai

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
from openai import OpenAI
from google.colab import userdata
import os

In [ ]:
# Set the API key from Colab's user data secrets
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

client = OpenAI()

client.batches.create(
  input_file_id="file-abc123",
  endpoint="/v1/chat/completions",
  completion_window="24h"
)

In [ ]:
# Don't need training data because LLM is pre-trained
# Just need validation set used in other models
new_summarized = pd.read_csv('/content/gdrive/MyDrive/DSSI 2025/Group 1: DSSI Summer 2025/Data/summarized_validation_set.csv')

Batch Prompting

In [ ]:
import openai
def classify_batch(batch_texts, start_idx=1):
    few_shot_prompt = """A Form 8-K is a public document that publicly traded companies must file with the Securities and Exchange Commission (SEC) to disclose significant, unscheduled events or corporate changes. These events are considered material, meaning they could impact the company's financial performance or investor decisions.
        Debt financing is when a company borrows money (or is given the right to borrow money) from a collection of lenders. This is primarily done through bonds, notes, loans, credit facilities. Key words include: Term Loan Agreement, Amended and Restated Credit Agreement, Revolving Credit Agreement, Junior Subordinated Notes, Bond, Debt Settlement Agreement.
        Classification is 'True' if the item contains details about a debt instrument. If the item just references a debt instrument, this column should still be false. There must be some details outside of the name (such as the start date, the amount, the lenders, etc.).
        I want you to think like a financial analyst reviewing SEC Form 8-K filings to determine whether they contain information related to commercial debt. Your response should classify each text with a “True” or “False” label. Do not explain.

Examples:
1. Text: "Credit Agreement  In connection with the completion of the Merger, on August 4, 2016, Higher One, Inc. ( Higher One ), a Delaware corporation and direct  wholly owned subsidiary of the Company, terminated the existing credit agreement, dated as of October 16, 2012 (as amended, the  Credit Agreement ), among Higher One, as borrower, the Company, Higher One Machines, Inc.,  Higher One Real Estate, Inc. and Higher One Real Estate SP, LLC, as guarantors, Bank of America, N.A. as administrative agent, swingline lender and letter of credit issuer, and the other lenders party thereto, and all commitments thereunder. At the  time of such termination, there were no outstanding revolving loans or outstanding letters of credit under the Credit Agreement. In connection with such termination, all liens on the assets of Higher One, the Company and Higher Ones  subsidiaries securing such facility, together with such subsidiary or Company guarantees, were released and terminated."
   True
2. Text: "Alpha Holding Company, LLC Disposition  On January 19, 2017, Concho Resources Inc. (the  Company ), through its wholly owned subsidiary, COG Operating LLC  ( Seller ), entered into a definitive securities purchase agreement (the  Purchase Agreement ) with Plains Pipeline, L.P. ( Purchaser ) to sell 100% of the issued and  outstanding Class A Units of Alpha Holding Company, LLC ( AHC ) (the  Disposition ), for aggregate consideration of approximately $800 million, subject to customary purchase price  adjustments, after closing of the transactions contemplated by the Class B Purchase Agreement (defined below) and repayment of the outstanding borrowings under the credit facility of Alpha Crude Connector, LLC, a wholly owned subsidiary of AHC.  Contemporaneously with the execution of the Purchase Agreement, Purchaser entered into a definitive purchase agreement with Frontier Midstream Solutions, LLC ( Frontier ) to purchase 100% of the issued and outstanding  Class B Units of AHC (the  Class  B Purchase Agreement ). Closing under the Purchase Agreement and the Class B Purchase Agreement is anticipated to occur contemporaneously during the  first half of 2017, subject to customary closing conditions, as further described below. Seller retains the right under the Limited Liability Company Agreement of AHC, subject to certain conditions, to purchase or cause AHC to purchase all of the  Class B Units of AHC prior to closing under the Purchase Agreement and the Class B Purchase Agreement (an  Option Exercise ). In the event of an Option Exercise, Seller will sell all of the Class A Units and  the Class B Units of AHC to Purchaser for an aggregate purchase price of approximately $1.215 billion. The Company expects the net proceeds to Seller in the event of an Option Exercise and sale to Purchaser of all of the Class A Units and  the Class B Units of AHC to be substantially similar to what Seller would otherwise receive under the Purchase Agreement in the absence of an Option Exercise.  Each partys obligation to consummate the Disposition is conditioned upon, among other things, (i) confirmation of the  counterparties representations and warranties as of the closing, (ii) the counterparties performance, in all material respects, of all covenants, (iii) the receipt of all required approvals (including approval under the  Hart-Scott-Rodino Antitrust Improvements Act of 1976), (iv) the absence of legal matters prohibiting the Disposition, and (v) the satisfaction or waiver by Frontier or Purchaser, as applicable, of certain conditions precedent to the  consummation of the transactions contemplated by the Class B Purchase Agreement.  "
   False
3. Text: "Between December 14, 2022 and December 16, 2022, IronNet, Inc. (the " Company ") issued and sold secured promissory notes in an aggregate principal amount of $6.9 million (the " Notes ") to a total of eight lenders. The lenders included GEN Keith B. Alexander (Ret.), the founder, Chief Executive Officer and Chairman of the Board of Directors of the Company. The Notes were also issued to lenders that include VADM John M. McConnell (Ret.), Hon. Michael J. Rogers, Theodore E. Schlein and VADM Jan E. Tighe (Ret.), each of whom is a director of the Company, as well as funds affiliated with Forgepoint Capital (the " Forgepoint Funds "). Donald R. Dixon, a member of the Company's board of directors, is an affiliate of ForgePoint Capital, and the ForgePoint Funds are also beneficial owners of greater than 5% of the Company's common stock.    Each Note has a scheduled maturity date of one year from the date of issuance, except that the Note issued to one lender that is not, and is not affiliated with, a director of the Company has a scheduled maturity date of six months from the date of issuance. The Notes bear interest at a rate of 13.8% per annum, payable at maturity. The Company may prepay the Notes at any time without premium or penalty, and the Notes do not restrict the incurrence of future indebtedness by the Company.    In the event that the Company fails to timely pay amounts due under the Notes or the Company materially defaults in its performance of any other covenant under the Notes or the Security Agreement (as defined below), which default is not cured within 30 days after written notice thereof, then at the option of the lenders, upon the written consent of lenders representing a majority of the outstanding and unpaid principal under the Notes, all unpaid principal, accrued interest and other amounts owing under the Notes shall be immediately due and payable.    In the event that the Company files any petition or action for relief under any bankruptcy, reorganization, insolvency or moratorium law or any other law for the relief of or relating to debtors, or makes any assignment for the benefit of creditors or takes any corporate action in furtherance thereof, or if an involuntary petition is filed against the Company (unless such petition is dismissed or discharged within 60 days) under any bankruptcy statute, or a custodian, receiver, trustee, assignee for the benefit of creditors is appointed to take possession, custody or control of any property of the Company, then all unpaid principal, accrued interest and other amounts owing under the Notes will accelerate and automatically become immediately due and payable.    The Notes are secured by substantially all of the assets of the Company, excluding the Company's intellectual property. The Company entered into a Security Agreement (the " Security Agreements ") with each of the lenders.    The forgoing descriptions of the Notes and the Security Agreements do not purport to be complete and are subject to, and are qualified in their entirety by reference to, the form of Note and form of Security Agreement filed hereto as Exhibits 4.1 and 10.1, respectively, and incorporated herein by reference.  "
   True

Now classify the following texts:
"""
    numbered_batch = ""
    for i, sentence in enumerate(batch_texts, start=start_idx):
        numbered_batch += f"{i}. \"{sentence}\" →\n"

    final_prompt = few_shot_prompt + numbered_batch

    response = client.chat.completions.create(
        model="gpt-3.5-turbo", # Faster than GPT-4; Went from 6-7 minutes (only a third of it completed) to 1 minute (fully completed)
        messages=[{"role": "user", "content": final_prompt}],
        temperature=0.10
    )

    return response.choices[0].message.content

results = []
batch_size = 5 # Process 5 rows at a time

In [ ]:
# Convert the text data into a list, loop through it in batches, classify each batch,
texts = new_summarized['text'].astype(str).tolist()

results = [] # Initialize the results list before the loop

for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    output = classify_batch(batch, start_idx=i+1)
    #print(f"Batch {i//batch_size + 1} output:")
    print(output)
    # Extract just the True/False from each line of the output
    batch_results = [r.split('. ')[1] for r in output.strip().split('\n')]
    results.extend(batch_results)

# Print the output, and collect all classification results in one list
print(f"Total results collected: {len(results)}")

In [ ]:
print(new_summarized.columns)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Convert predictions (from model output strings) to booleans
preds = [r.strip().lower() == 'true' for r in results]

# Get true labels and convert to booleans
true = [str(label).strip().lower() == 'true' for label in new_summarized['contains_debt_instrument_information']]

# Classification metrics
accuracy = accuracy_score(true, preds)
precision = precision_score(true, preds)
recall = recall_score(true, preds)
f1 = f1_score(true, preds)

print("Accuracy:", round(accuracy, 4))
print("Precision:", round(precision, 4))
print("Recall:", round(recall, 4))
print("F1 Score:", round(f1, 4))


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Create confusion matrix
cm = confusion_matrix(true, preds)

# Plot
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['False', 'True'])
disp.plot(cmap='Blues')
plt.title('GPT 3.5 Turbo Confusion Matrix: Summarized Dataset')
plt.tight_layout()
plt.show()

If I wanted a classification report:

from sklearn.metrics import classification_report

print("\nClassification Report:")

print(classification_report(true, preds))

In [ ]:
from wordcloud import WordCloud
import random
custom_colors = ["#4C69B2", "#FF4949", "#FF7A78", "#FF9D35", "#6389EB", "#FFC53A"]
def random_color_func(word=None, font_size=None, position=None, orientation=None, font_path=None, random_state=None):
    return random.choice(custom_colors)

# Add 'preds' and 'true' columns to the DataFrame
new_summarized['preds'] = preds
new_summarized['true'] = true

# Subsets
fp = new_summarized[(new_summarized["preds"] == 1) & (new_summarized["true"] == 0)]
fn = new_summarized[(new_summarized["preds"] == 0) & (new_summarized["true"] == 1)]

# Combine all FP/FN texts
fp_text = " ".join(fp['text'])
fn_text = " ".join(fn['text'])

# Generate word clouds with custom color
wordcloud_fp = WordCloud(width=800, height=400, background_color='white', color_func=random_color_func).generate(fp_text)
wordcloud_fn = WordCloud(width=800, height=400, background_color='white', color_func=random_color_func).generate(fn_text)

# Plot
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(wordcloud_fp, interpolation='bilinear')
plt.title('False Positives')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(wordcloud_fn, interpolation='bilinear')
plt.title('False Negatives')
plt.axis('off')
plt.show()

In [ ]:
# Save as CSV
output_path = '/content/gdrive/MyDrive/DSSI 2025/Group 1: DSSI Summer 2025/Data/Prompt Engineering Datasets/gpt3.5_summarized_recent.csv'
new_summarized.to_csv(output_path, index=False)